In [5]:
import os
import pandas as pd
import numpy as np

from decimal import Decimal

from IPython.display import display

In [6]:
def time_convert(x):
    if isinstance(x, int):
        return x
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

In [7]:
data_path = os.path.join('ikz_pld_plugin','tests','data')

In [8]:
with open(os.path.join(data_path,'26042023_1630-STO-SAO-STO-Alev.elog'), 'r') as f:
    df_recipe = pd.read_csv(
        f,
        sep='\t',
        names=['time_h','process'],
        header=None,
    )
df_recipe['time_s'] = df_recipe['time_h'].apply(time_convert)
df_recipe['duration_s'] = df_recipe['time_s'].diff(-1) * -1
df_steps = df_recipe.iloc[1:-1:3,:].copy()
df_steps['pulses'] = df_recipe.iloc[2:-1:3,1].str.split().str[0].values
df_steps['recipe'] = df_steps['process'].str.split(':').str[1]
display(df_recipe)
display(df_steps)

,time_h,process,time_s,duration_s
0,0:0:14,Starting Evacuation Recipe,14,617.0
1,0:10:31,Starting Process with Recipe :heatSAO500,631,1515.0
2,0:35:46,0 Laser-pulse fired on target,2146,-0.0
3,0:35:46,Event completed,2146,16.0
4,0:36:2,Starting Process with Recipe :heatSAO700,2162,1272.0
5,0:57:14,0 Laser-pulse fired on target,3434,-0.0
6,0:57:14,Event completed,3434,16.0
7,0:57:30,Starting Process with Recipe :dwellSAO700,3450,600.0
8,1:7:30,0 Laser-pulse fired on target,4050,-0.0
9,1:7:30,Event completed,4050,18.0


,time_h,process,time_s,duration_s,pulses,recipe
1,0:10:31,Starting Process with Recipe :heatSAO500,631,1515.0,0,heatSAO500
4,0:36:2,Starting Process with Recipe :heatSAO700,2162,1272.0,0,heatSAO700
7,0:57:30,Starting Process with Recipe :dwellSAO700,3450,600.0,0,dwellSAO700
10,1:7:48,Starting Process with Recipe :depoSAO700,4068,174.0,330,depoSAO700
13,1:11:0,Starting Process with Recipe :dwellSTO700,4260,300.0,0,dwellSTO700
16,1:16:18,Starting Process with Recipe :depoSTO700,4578,159.0,300,depoSTO700
19,1:19:18,Starting Process with Recipe :coolSTO500,4758,1479.0,0,coolSTO500
22,1:44:13,Starting Process with Recipe :coolSTO20,6253,9908.0,0,coolSTO20


In [16]:
columns = [
    'time_s',
    'temperature_degc',
    'pressure2_mbar',
    'o2_flow_sccm',
    'n2_ar_flow_sccm',
    'frequency_hz',
    'laser_energy_mj',
    'pressure1_mbar',
    'zeros',
]
with open(os.path.join(data_path,'26042023_1630-STO-SAO-STO-Alev.dlog'), 'r') as d_log:
    df_data = pd.read_csv(
        d_log,
        sep='\t',
        names=columns
    )

attenuated_laser_energy = 50e-3  # Joule
steps = []
for idx, row in df_steps.iterrows():
    data = df_data.loc[
        (row['time_s'] <= df_data['time_s']) & 
        (df_data['time_s'] < (row['time_s'] + row['duration_s']))
    ].copy()
    data['pressure_mbar'] = data['pressure1_mbar']
    p2_range = (0.01 <= data['pressure_mbar']) & (data['pressure_mbar'] <= 0.1)
    data.loc[p2_range, 'pressure_mbar'] = data.loc[p2_range, 'pressure2_mbar']
    mean_laser_energy = data['laser_energy_mj'].replace(0, np.NaN).mean()
    if np.isnan(mean_laser_energy):
        attenuation = 1
    else:
        attenuation = attenuated_laser_energy / mean_laser_energy
    data['scaled_laser_energy_mj'] = data['laser_energy_mj'] * attenuation
    print(f"{np.isnan(mean_laser_energy) = }")
    # print(f"{data[data['laser_energy_mj'] > 0, 'laser_energy_mj'].mean() = }")
    print(f"{data['frequency_hz'].max() = }")
    print(f"{data['frequency_hz'].min() = }")
    print(f"{data['frequency_hz'].mean() = }")
    display(data)

np.isnan(mean_laser_energy) = True
data['frequency_hz'].max() = 0.0
data['frequency_hz'].min() = 0.0
data['frequency_hz'].mean() = 0.0


,time_s,temperature_degc,pressure2_mbar,o2_flow_sccm,n2_ar_flow_sccm,frequency_hz,laser_energy_mj,pressure1_mbar,zeros,pressure_mbar
213,631.4,30.8,0.3833,79.32,0.0,0.0,0.0,0.3346,NaN,0.3346
214,633.5,30.9,0.4146,78.64,0.0,0.0,0.0,0.3640,NaN,0.3640
215,635.6,31.0,0.4441,78.06,0.0,0.0,0.0,0.3896,NaN,0.3896
216,637.6,31.0,0.4667,76.64,0.0,0.0,0.0,0.4093,NaN,0.4093
217,639.8,31.0,0.4883,75.81,0.0,0.0,0.0,0.4284,NaN,0.4284
...,...,...,...,...,...,...,...,...,...,...
938,2137.0,505.5,0.2000,12.11,0.0,0.0,0.0,0.1803,NaN,0.1803
939,2139.0,505.5,0.2000,12.16,0.0,0.0,0.0,0.1803,NaN,0.1803
940,2141.0,505.4,0.2000,12.11,0.0,0.0,0.0,0.1803,NaN,0.1803
941,2143.0,505.2,0.2000,12.16,0.0,0.0,0.0,0.1803,NaN,0.1803


np.isnan(mean_laser_energy) = True
data['frequency_hz'].max() = 0.0
data['frequency_hz'].min() = 0.0
data['frequency_hz'].mean() = 0.0


,time_s,temperature_degc,pressure2_mbar,o2_flow_sccm,n2_ar_flow_sccm,frequency_hz,laser_energy_mj,pressure1_mbar,zeros,pressure_mbar
943,2162.0,503.9,0.18120,7.130,0.0,0.0,0.0,0.16240,NaN,0.16240
944,2164.0,503.8,0.17840,6.788,0.0,0.0,0.0,0.16010,NaN,0.16010
945,2167.0,503.6,0.17580,6.593,0.0,0.0,0.0,0.15740,NaN,0.15740
946,2169.0,503.6,0.17310,6.495,0.0,0.0,0.0,0.15520,NaN,0.15520
947,2171.0,503.5,0.17060,6.251,0.0,0.0,0.0,0.15280,NaN,0.15280
...,...,...,...,...,...,...,...,...,...,...
1549,3423.0,713.1,0.09997,4.639,0.0,0.0,0.0,0.09031,NaN,0.09997
1550,3425.0,713.6,0.09997,4.590,0.0,0.0,0.0,0.09031,NaN,0.09997
1551,3427.0,714.0,0.09997,4.639,0.0,0.0,0.0,0.09020,NaN,0.09997
1552,3429.0,714.3,0.09997,4.639,0.0,0.0,0.0,0.09031,NaN,0.09997


np.isnan(mean_laser_energy) = True
data['frequency_hz'].max() = 0.0
data['frequency_hz'].min() = 0.0
data['frequency_hz'].mean() = 0.0


...

,time_s,temperature_degc,pressure2_mbar,o2_flow_sccm,n2_ar_flow_sccm,frequency_hz,laser_energy_mj,pressure1_mbar,zeros,pressure_mbar
2069,4579.0,749.5,0.09977,4.639,0.0,2.0,0.0,0.09052,NaN,0.09977
2070,4581.0,749.6,0.09973,4.639,0.0,2.0,0.0,0.09052,NaN,0.09973
2071,4583.0,749.5,0.09977,4.639,0.0,2.0,0.0,0.09063,NaN,0.09977
2072,4585.0,749.5,0.09977,4.639,0.0,2.0,0.0,0.09063,NaN,0.09977
2073,4587.0,749.5,0.09981,4.639,0.0,2.0,400.3,0.09063,NaN,0.09981
...,...,...,...,...,...,...,...,...,...,...
2139,4728.0,749.5,0.10010,4.639,0.0,2.0,400.0,0.09105,NaN,0.10010
2140,4730.0,749.5,0.10010,4.639,0.0,2.0,400.5,0.09105,NaN,0.10010
2141,4732.0,749.5,0.10010,4.639,0.0,2.0,400.4,0.09105,NaN,0.10010
2142,4734.0,749.5,0.10010,4.639,0.0,2.0,400.5,0.09116,NaN,0.10010


np.isnan(mean_laser_energy) = True
data['frequency_hz'].max() = 0.0
data['frequency_hz'].min() = 0.0
data['frequency_hz'].mean() = 0.0


,time_s,temperature_degc,pressure2_mbar,o2_flow_sccm,n2_ar_flow_sccm,frequency_hz,laser_energy_mj,pressure1_mbar,zeros,pressure_mbar
2144,4759.0,749.5,0.1086,81.33,0.0,0.0,0.0,0.09475,NaN,0.1086
2145,4762.0,749.6,0.1988,98.91,0.0,0.0,0.0,0.18390,NaN,0.1839
2146,4764.0,750.0,0.2847,99.69,0.0,0.0,0.0,0.26190,NaN,0.2619
2147,4766.0,750.2,0.3695,99.74,0.0,0.0,0.0,0.33810,NaN,0.3381
2148,4768.0,750.5,0.4657,99.84,0.0,0.0,0.0,0.41900,NaN,0.4190
...,...,...,...,...,...,...,...,...,...,...
2850,6227.0,506.5,109.8000,99.88,0.0,0.0,0.0,109.80000,NaN,109.8000
2851,6229.0,506.0,110.1000,99.84,0.0,0.0,0.0,110.10000,NaN,110.1000
2852,6231.0,505.8,110.6000,99.88,0.0,0.0,0.0,110.60000,NaN,110.6000
2853,6234.0,505.5,111.0000,99.84,0.0,0.0,0.0,111.00000,NaN,111.0000


np.isnan(mean_laser_energy) = True
data['frequency_hz'].max() = 0.0
data['frequency_hz'].min() = 0.0
data['frequency_hz'].mean() = 0.0


,time_s,temperature_degc,pressure2_mbar,o2_flow_sccm,n2_ar_flow_sccm,frequency_hz,laser_energy_mj,pressure1_mbar,zeros,pressure_mbar
2855,6253.0,502.5,111.5,1.9530,0.0,0.0,0.0,111.5,NaN,111.5
2856,6255.0,502.1,111.7,82.9900,0.0,0.0,0.0,111.7,NaN,111.7
2857,6257.0,501.9,112.0,98.8100,0.0,0.0,0.0,112.0,NaN,112.0
2858,6259.0,501.6,112.6,99.6900,0.0,0.0,0.0,112.6,NaN,112.6
2859,6261.0,501.5,112.8,99.7900,0.0,0.0,0.0,112.8,NaN,112.8
...,...,...,...,...,...,...,...,...,...,...
7621,16150.0,25.0,184.1,0.7813,0.0,0.0,0.0,184.1,NaN,184.1
7622,16150.0,25.1,184.1,0.8302,0.0,0.0,0.0,184.1,NaN,184.1
7623,16160.0,25.0,184.1,0.7813,0.0,0.0,0.0,184.1,NaN,184.1
7624,16160.0,25.1,184.1,0.7813,0.0,0.0,0.0,184.1,NaN,184.1


In [19]:
#function to convert hh:mm:ss to seconds to make the values consistent with dlog data
def time_convert(x):
    if isinstance(x, int):
        return x
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

df_recipe['Time']=df_recipe.Time.apply(time_convert)
df_recipe


,Time,Process,Laser on
0,631,heatSAO500,False
3,2162,heatSAO700,False
6,3450,dwellSAO700,False
9,4068,depoSAO700,True
12,4260,dwellSTO700,False
15,4578,depoSTO700,True
18,4758,coolSTO500,False
21,6253,coolSTO20,False
24,16161,NaN,NaN


In [164]:
depositions = df_recipe.loc[df_recipe['Laser on'] == True]
depo_start = depositions['Time'].values.tolist() #gets deposition start times
depo_names = depositions['Process'].values.tolist() #gets recipe names of depositions
depo_names

['depoSAO700', 'depoSTO700']

In [1]:
colnames = ['Time [s]','Temperature [°C]','Pressure2 [mbar]', 'O2 Flow [sccm]', 'N2/Ar Flow [sccm]','Frequency [Hz]', 
          'Laserenergy [mJ]','Pressure1 [mbar]','Zeros']
with open('PLD_dlog.dlog', 'r') as dlog:
    df = pd.read_csv(dlog, sep='\t', names=colnames) #creating data frame from dlog files
dlog.close()
df

FileNotFoundError: [Errno 2] No such file or directory: 'PLD_dlog.dlog'

In [169]:
growthcons = df.loc[df['Laserenergy [mJ]'] > 0.0] #creating a data frame with growth conditions where laser was turned on

mean_con = []

#averaging data of the single depostion steps and putting the values in a data frame
#sequenced in reversed order for easier splitting of depostion steps
for x in reversed(range(len(depo_start))): 
    growthcon_step = growthcons[growthcons['Time [s]'] >= depo_start[x]]
    step = []
    
    step.append(depo_names[x])
    step.append(round(growthcon_step['Temperature [°C]'].mean(),1))    
    step.append(f"{Decimal(growthcon_step['Pressure1 [mbar]'].mean()):.1E}")
    step.append(f"{Decimal(growthcon_step['Pressure2 [mbar]'].mean()):.1E}")
    freq = growthcon_step['Frequency [Hz]'].mean()
    step.append(freq)
    step.append(round(growthcon_step['Laserenergy [mJ]'].mean(),1))

    step_time = growthcon_step['Time [s]'].iloc[-1] - growthcon_step['Time [s]'].iloc[0]
    step.append(step_time)
    step.append(freq*step_time)
    mean_con.append(step)
    growthcons = growthcons[growthcons['Time [s]'] < depo_start[x]]

avg_growthcons = pd.DataFrame(mean_con,columns = ['Recipe name','Temperature [°C]','Pressure1 [mbar]','Pressure2 [mbar]',
                                                  'Frequency [Hz]', 'Laserenergy [mJ]','Step time [s]','Lasershots'])
avg_growthcons = avg_growthcons[::-1] #reversing the order again to represent correct order
avg_growthcons.index = avg_growthcons.index.values[::-1]
avg_growthcons
    

,Recipe name,Temperature [°C],Pressure1 [mbar],Pressure2 [mbar],Frequency [Hz],Laserenergy [mJ],Step time [s],Lasershots
0,depoSAO700,749.8,9.1E-2,1.0E-1,2.0,400.1,162.0,324.0
1,depoSTO700,749.4,9.1E-2,1.0E-1,2.0,400.0,147.0,294.0


In [6]:
import re
import datetime

pattern = re.compile(
    r'(?P<datetime>\d{8}_\d{4})-(?P<name>.+)\.(?P<type>d|e)log'
)
match = pattern.match('26042023_1630-STO-SAO-STO-Alev.dlog')
test = datetime.datetime.strptime(match['datetime'], r'%d%m%Y_%H%M')
print(test)
print(test + datetime.timedelta(seconds=10))
display(test)
from zoneinfo import ZoneInfo
print(test.astimezone())

2023-04-26 16:30:00
2023-04-26 16:30:10


datetime.datetime(2023, 4, 26, 16, 30)

2023-04-26 16:30:00+02:00


In [9]:
import datetime

print(datetime.datetime.now().isoformat())

2023-06-16T17:46:27.744391
